<a href="https://colab.research.google.com/github/Nunuy15/Thesis/blob/main/EfficientNetB0_Uji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Directories
train_dir = '/content/drive/MyDrive/DATASET_FFB/SUHARJITO/NORMAL/DATASET_PHOTOMETRIC/train'
val_dir = '/content/drive/MyDrive/DATASET_FFB/SUHARJITO/NORMAL/DATASET_PHOTOMETRIC/valid'
test_dir = '/content/drive/MyDrive/DATASET_FFB/SUHARJITO/NORMAL/DATASET_PHOTOMETRIC/test'

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# Hyperparameters
learning_rate = 0.0001
momentum = 0.9
batch_size = 8
epochs = 20
image_size = (224, 224)

# Data preprocessing
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
base_model.trainable = False

# Build classification model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(6, activation='softmax')(x)  # 6 classes

model = Model(inputs=base_model.input, outputs=output)

# Compile model
optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Save the trained model
model.save('/content/drive/MyDrive/DATASET_FFB/SUHARJITO/NORMAL/DATASET_PHOTOMETRIC/efficientnetb0_kelapa_sawit_classification.keras')


Found 10710 images belonging to 6 classes.
Found 3060 images belonging to 6 classes.
16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/20
1339/1339 [==============================] - 2746s 2s/step - loss: 1.3764 - accuracy: 0.5120 - val_loss: 1.1328 - val_accuracy: 0.6072
Epoch 2/20
1339/1339 [==============================] - 181s 135ms/step - loss: 1.0537 - accuracy: 0.6322 - val_loss: 0.9742 - val_accuracy: 0.6520
Epoch 3/20
1339/1339 [==============================] - 178s 133ms/step - loss: 0.9257 - accuracy: 0.6718 - val_loss: 0.8885 - val_accuracy: 0.6824
Epoch 4/20
1339/1339 [==============================] - 181s 135ms/step - loss: 0.8585 - accuracy: 0.6923 - val_loss: 0.8360 - val_accuracy: 0.7000
Epoch 5/20
1339/1339 [==============================] - 177s 132ms/step - loss: 0.8068 - accuracy: 0.7078 - val_loss: 0.7919 - val_accuracy: 0.7154
Epoch 6/20
1339/1339 [==============================] - 180s 134ms/step - loss: 0.7629 - accuracy: 0.7260 - val_

In [3]:
# Test data generator
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Found 1530 images belonging to 6 classes.
192/192 [==============================] - 310s 2s/step - loss: 0.5233 - accuracy: 0.7974
Test Accuracy: 79.74%


# AUGMENTASI

In [ ]:
!pip install albumentations

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import albumentations as A
from albumentations.core.composition import Compose
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os

# Hyperparameters
learning_rate = 0.0001
momentum = 0.9
batch_size = 8
epochs = 150
image_size = (224, 224)

# Directories
train_dir = '/content/drive/MyDrive/DATASET_FFB/SUHARJITO/NORMAL/DATASET_BALANCED_SPLIT_MIN/train'
val_dir = '/content/drive/MyDrive/DATASET_FFB/SUHARJITO/NORMAL/DATASET_BALANCED_SPLIT_MIN/valid'
test_dir = '/content/drive/MyDrive/DATASET_FFB/SUHARJITO/NORMAL/DATASET_BALANCED_SPLIT_MIN/test'

# Augmentation function
def custom_augmentation(image):
    # Define augmentations
    transform = Compose([
        A.RandomCrop(width=image_size[0] - 20, height=image_size[1] - 20, always_apply=True),  # Localization
        A.Resize(height=image_size[0], width=image_size[1]),  # Resize after cropping
        A.GaussianBlur(blur_limit=(3, 7), p=0.5),  # Gaussian Blur
        A.Rotate(limit=40, p=1.0)  # 9-angle crop simulation through rotation
    ])
    augmented = transform(image=image)
    return augmented['image'] / 255.0  # Normalize to [0, 1] range manually

# Custom Data Generator with Albumentations
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory, batch_size, image_size, aug_func, class_mode='categorical'):
        self.directory = directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.aug_func = aug_func
        self.class_mode = class_mode
        self.class_indices = {v: k for k, v in enumerate(sorted(os.listdir(directory)))}
        self.image_paths = []
        self.labels = []
        for class_name, idx in self.class_indices.items():
            class_path = os.path.join(directory, class_name)
            self.image_paths += [os.path.join(class_path, img) for img in os.listdir(class_path)]
            self.labels += [idx] * len(os.listdir(class_path))

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_x = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = []
        labels = []

        for img_path, label in zip(batch_x, batch_y):
            image = load_img(img_path, target_size=self.image_size)
            image = img_to_array(image)
            image = self.aug_func(image)  # Apply augmentations without keyword argument
            images.append(image)
            labels.append(label)

        images = np.array(images, dtype="float32")
        labels = tf.keras.utils.to_categorical(labels, num_classes=6)  # Adjust for 6 classes

        return images, labels

# Initialize data generators
train_generator = CustomDataGenerator(
    directory=train_dir,
    batch_size=batch_size,
    image_size=image_size,
    aug_func=custom_augmentation  # Augmentasi diterapkan pada data train
)

validation_generator = CustomDataGenerator(
    directory=val_dir,
    batch_size=batch_size,
    image_size=image_size,
    aug_func=lambda x: x / 255.0  # Hanya normalisasi, tanpa keyword argument
)

test_generator = CustomDataGenerator(
    directory=test_dir,
    batch_size=batch_size,
    image_size=image_size,
    aug_func=lambda x: x / 255.0  # Hanya normalisasi, tanpa keyword argument
)

# Load EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
base_model.trainable = False

# Build classification model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(6, activation='softmax')(x)  # 6 classes

model = Model(inputs=base_model.input, outputs=output)

# Compile model
optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Save the trained model
model.save('/content/drive/MyDrive/path/to/saved_model/efficientnetb0_kelapa_sawit_classification_with_aug.keras')


Epoch 1/150
514/514 ━━━━━━━━━━━━━━━━━━━━ 366s 677ms/step - accuracy: 0.1575 - loss: 1.7948 - val_accuracy: 0.1602 - val_loss: 1.7921
Epoch 2/150
514/514 ━━━━━━━━━━━━━━━━━━━━ 381s 677ms/step - accuracy: 0.1878 - loss: 1.7909 - val_accuracy: 0.1699 - val_loss: 1.7920
Epoch 3/150
514/514 ━━━━━━━━━━━━━━━━━━━━ 380s 674ms/step - accuracy: 0.1621 - loss: 1.7924 - val_accuracy: 0.1602 - val_loss: 1.7919
Epoch 4/150
514/514 ━━━━━━━━━━━━━━━━━━━━ 338s 657ms/step - accuracy: 0.1505 - loss: 1.7933 - val_accuracy: 0.1680 - val_loss: 1.7919
Epoch 5/150
514/514 ━━━━━━━━━━━━━━━━━━━━ 336s 652ms/step - accuracy: 0.1877 - loss: 1.7921 - val_accuracy: 0.1582 - val_loss: 1.7918
Epoch 6/150
514/514 ━━━━━━━━━━━━━━━━━━━━ 375s 640ms/step - accuracy: 0.1456 - loss: 1.7929 - val_accuracy: 0.1680 - val_loss: 1.7918
Epoch 7/150
514/514 ━━━━━━━━━━━━━━━━━━━━ 336s 652ms/step - accuracy: 0.1927 - loss: 1.7917 - val_accuracy: 0.1582 - val_loss: 1.7918
Epoch 8/150
514/514 ━━━━━━━━━━━━━━━━━━━━ 393s 675ms/step - accuracy: 

In [ ]:
# Test Data Generator
test_datagen = CustomDataGenerator(
    directory=test_dir,
    batch_size=batch_size,
    image_size=image_size,
    aug_func=lambda x: x / 255.0  # No augmentations, only normalization
)

# Load the saved model
model = tf.keras.models.load_model('efficientnetb0_kelapa_sawit_classification_with_aug.keras')

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_datagen)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Loss: {test_loss:.4f}')


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the saved model
model = tf.keras.models.load_model('efficientnetb0_kelapa_sawit_classification_with_aug.keras')

# Define custom generators for training, validation, and test data
train_datagen = CustomDataGenerator(
    directory=train_dir,
    batch_size=batch_size,
    image_size=image_size,
    aug_func=lambda x: x / 255.0  # No augmentations, only normalization
)

validation_datagen = CustomDataGenerator(
    directory=val_dir,
    batch_size=batch_size,
    image_size=image_size,
    aug_func=lambda x: x / 255.0
)

test_datagen = CustomDataGenerator(
    directory=test_dir,
    batch_size=batch_size,
    image_size=image_size,
    aug_func=lambda x: x / 255.0
)

# Evaluate on train, validation, and test sets
train_loss, train_accuracy = model.evaluate(train_datagen)
val_loss, val_accuracy = model.evaluate(validation_datagen)
test_loss, test_accuracy = model.evaluate(test_datagen)

# Tabel Akurasi
accuracy_df = pd.DataFrame({
    "Dataset": ["Train", "Validation", "Test"],
    "Loss": [train_loss, val_loss, test_loss],
    "Accuracy": [train_accuracy, val_accuracy, test_accuracy]
})

# Display the accuracy table
import ace_tools as tools; tools.display_dataframe_to_user(name="Akurasi Dataset", dataframe=accuracy_df)

# Confusion Matrix for Test Set
# Generate predictions and true labels
y_pred = []
y_true = []

for images, labels in test_datagen:
    predictions = model.predict(images)
    y_pred.extend(np.argmax(predictions, axis=1))
    y_true.extend(np.argmax(labels, axis=1))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
class_labels = list(test_datagen.class_indices.keys())

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix for Test Set")
plt.show()

# Print classification report
report = classification_report(y_true, y_pred, target_names=class_labels)
print("Classification Report:\n", report)
